In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5
eps = np.finfo(float).eps

In [2]:
import sys
import time

In [3]:
import cv2

### Load backprojection library

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC/backprojection")

In [5]:
from loadbackprojection import *

In [6]:
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"

In [7]:
lib = LibBackProjection( libraryFilename )

### Load other tools

In [8]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")

In [9]:
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.disp_PoSAR_img import OPTt
from posarutils.process.filtering import box_filter

### Load parameters specific to the dataset

In [10]:
from rsc.datasetconfig import *

%load_ext autoreload
%autoreload 2

In [11]:
withPlots = 0

## Read parameters from the XML file

In [12]:
from posarutils.other.PosarMCParameters import *

In [13]:
data_date = "2018_05_29_15_09_37"
data_dir = "/home/pleroy/DATA/PoSAR-v2_PIMA_TEST-3/2018_05_29/" + data_date

In [14]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
Tp = params.configuredTRamp / 1e6
B0 = params.frequencyBand
fs = params.samplingFrequency
c = 3e8

## Load the analytic signal

In [15]:
rampUp = 0
withHanning = 0

In [16]:
firstFile = 71
nbFiles = 60
lastFile = firstFile + nbFiles - 1
firstRamp = (firstFile) * params.rampsPerFile
lastRamp = (lastFile) * params.rampsPerFile
if withHanning:
    if rampUp:
        RD1 = np.load( data_dir + '/RD_files_{}_{}_hanning.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}_hanning.npy'.format( firstFile, lastFile ) )
    else:
        RD1 = np.load( data_dir + '/RD_files_{}_{}_rampDown_hanning.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}_rampDown_hanning.npy'.format( firstFile, lastFile ) )
else:
    if rampUp:
        RD1 = np.load( data_dir + '/RD_files_{}_{}.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}.npy'.format( firstFile, lastFile ) )
    else:
        RD1 = np.load( data_dir + '/RD_files_{}_{}_rampDown.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}_rampDown.npy'.format( firstFile, lastFile ) )

# remove coupling from RD1 to build RD2
RD2 = RD1 - coupling
nbPos = params.rampsPerFile * nbFiles

In [17]:
if withPlots:
    plt.figure()
    plt.plot(np.abs(coupling))
    plt.grid()

## Load antenna positions

In [18]:
# load positions for all ramps
if rampUp == 1:
    filename = data_dir + "/rampNumber_timeStamp_xyz_rampUp.npy"
    xyz = np.load( filename )
else:
    filename = data_dir + "/rampNumber_timeStamp_xyz.npy"
    xyz = np.load( filename )
xa = xyz[:,2]
ya = xyz[:,3]
za = xyz[:,4]
xa_mean = np.mean(xa)
ya_mean = np.mean(ya)
za_mean = np.mean(za)
print( "xa_mean = {:.2f}, ya_mean = {:.2f}, za_mean = {:.2f}".format( xa_mean, ya_mean, za_mean ) )

# load scene elements coordinates
runaway_proj = np.load( data_dir + "/runaway_proj.npy" )
hangar_proj = np.load( data_dir + "/hangar_proj.npy" )
building_proj = np.load( data_dir + "/building_proj.npy" )
track_proj = np.load( data_dir + "/track_proj.npy" )
track_selection_proj = np.load( data_dir + "/track_selection_proj.npy" )
cornerReflectorLarge_proj = np.load( data_dir + "/cornerReflectorLarge_proj.npy" )
cornerReflectorSmall_proj = np.load( data_dir + "/cornerReflectorSmall_proj.npy" )

xa_mean = -1023.80, ya_mean = -172.45, za_mean = 418.68


In [19]:
print( "RD1.shape = {}, xyz.shape = {}".format( RD1.shape, xyz.shape ) )

RD1.shape = (45000, 3000), xyz.shape = (102000, 5)


In [20]:
if withPlots == 1:
    plt.figure()

    title = data_date + " selection {} to {}".format( firstFile, lastFile )
    
    plt.suptitle( title )

    plt.subplot(221)
    plt.plot(xyz[:, 1], xyz[:, 2], label="x ")
    plt.plot(xyz[firstRamp:lastRamp, 1], xyz[firstRamp:lastRamp, 2], 'orange', label="x selection")
    plt.grid()
    plt.legend()

    plt.subplot(222)
    plt.plot(xyz[:, 1], xyz[:, 3], label="y ")
    plt.plot(xyz[firstRamp:lastRamp, 1], xyz[firstRamp:lastRamp, 3], 'orange', label="y selection")
    plt.grid()
    plt.legend()

    plt.subplot(223)
    plt.plot(xyz[:, 1], xyz[:, 4], label="z ")
    plt.plot(xyz[firstRamp:lastRamp, 1], xyz[firstRamp:lastRamp, 4], 'orange', label="z selection")
    plt.grid()
    plt.legend()

    plt.subplot(224)
    plt.plot(xyz[:, 2], xyz[:, 3], label="xy ")
    plt.plot(xyz[firstRamp:lastRamp, 2], xyz[firstRamp:lastRamp, 3], 'orange', label="xy (selection)")
    ax = plt.gca()
    ax.invert_xaxis()
    ax.invert_yaxis()
    ax.xaxis.tick_top()
    ax.yaxis.tick_right()
    plt.grid()
    plt.legend()
    
    plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

## Focalization

In [21]:
RD = np.fft.ifftshift(RD2, 1)

### Compute the aperture angle to have a specified resolution in azimuth

In [22]:
# dx = lambda_c / (4 * sin( phi_a / 2 )) => phi_a = 2 * asin( lambda_c / (4 * dx))
lambda_c = 3e8 / 5.8e9

phi_a = 60 * np.pi / 180
dx = lambda_c / (4 * np.sin( phi_a / 2 ))
print( "phi_a = {:.3f}, dx = {:.3f}".format(phi_a * 180 / np.pi, dx) )

phi = 20 * np.pi / 180
dx = lambda_c / (4 * np.sin( phi / 2 ))
print( "phi_a = {:.3f}, dx = {:.3f}".format(phi * 180 / np.pi, dx) )

phi_b = 1 * np.pi / 180
dx = lambda_c / (4 * np.sin( phi_b / 2 ))
print( "phi_b = {:.3f}, dx = {:.3f}".format(phi_b * 180 / np.pi, dx) )

dx2 = 1
phi_a2 = 2 * np.arcsin( lambda_c / (4 * dx2) )
print( "phi_a2 = {:.3f}, dx2 = {:.3f}".format(phi_a2 * 180 / np.pi, dx2) )

dx4 = 0.5
phi_a4 = 2 * np.arcsin( lambda_c / (4 * dx4) )
print( "phi_a2 = {:.3f}, dx2 = {:.3f}".format(phi_a4 * 180 / np.pi, dx4) )

dx3 = 0.25
phi_a3 = 2 * np.arcsin( lambda_c / (4 * dx3) )
print( "phi_a3 = {:.3f}, dx3 = {:.3f}".format(phi_a3 * 180 / np.pi, dx3) )


phi_a = 60.000, dx = 0.026
phi_a = 20.000, dx = 0.074
phi_b = 1.000, dx = 1.482
phi_a2 = 1.482, dx2 = 1.000
phi_a2 = 2.964, dx2 = 0.500
phi_a3 = 5.930, dx3 = 0.250


### Define the geometry of the scene

In [23]:
groundRange = 1 # 0 => slant range, 1 => ground range

# runaway
x_min = -500
x_max = 1000
y_min = -100
y_max = 400

x_min = -100
x_max = 500
y_min = -100
y_max = 400

hScene = 90

d_x = 1. # 0.1
d_y = 1. # 0.1

x = np.arange( x_min, x_max + d_x, d_x )
x = x.reshape( x.size )

# compute the distance from the scene
d_min = ( (ya_mean - y_min)**2 + (za_mean - hScene)**2 )**0.5
d_max = ( (ya_mean - y_max)**2 + (za_mean - hScene)**2 )**0.5
d = np.arange( d_min, d_max + d_y, d_y )
print("d.shape = {}, d_min = {:.2f}, d_max = {:.2f}".format(d.shape, np.amin(d), np.amax(d)))

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
if groundRange == 1:
    print("Ground range")
    extent = [ y_min, y_max, x_max, x_min ]
    im_extent = [ x_min, x_max, y_max, y_min ]
    y = np.arange( y_min, y_max + d_y, d_y )
else:
    print("Slant range")
    extent = [ d_min, d_max, x_max, x_min ]
    im_extent = [ x_min, x_max, d_max, d_min ]
    y = ya_mean + ( d**2 - (za_mean-hScene)**2 )**0.5
    
y = y.reshape( y.size )

d.shape = (325,), d_min = 336.57, d_max = 660.57
Ground range


In [24]:
sr = RD
Naz = sr.shape[0]
Nf = sr.shape[1]
overSamplingRatio = 10
Nover = overSamplingRatio * Nf
rangeResolution = c / (2 * B0)
r_base = np.arange( Nf ) * rangeResolution
r_over = np.arange( Nover ) * rangeResolution / overSamplingRatio
dr_over = r_over[1] - r_over[0]

print( "Nf = {}, Naz = {}".format( Nf, Naz ) )
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_over[0], r_over[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

Nf = 3000, Naz = 45000
range from 0.00m to 2999.90m, resolution = 1.0m, oversampled = 0.1m, 


In [25]:
lib.reload()

In [26]:
x.size, y.size

(601, 501)

In [27]:
myParameters = MyParameters()
myParameters.Nx = x.size
myParameters.Ny = y.size
myParameters.Nover = r_over.size
myParameters.dx = dr_over
myParameters.Naz = Naz
myParameters.Nf = Nf
myParameters.hScene = hScene

# resolution in azimuth
# 1.482° => 1m
# 60° => 2.6cm
# 20° => 7.4cm
# 5.930° => 25cm
myParameters.phi_a_deg = 20

In [28]:
t = time.time()

imgGroundRange  = np.zeros( (x.size, y.size), dtype=complex )
print( "img.shape = {}".format( imgGroundRange.shape ) )

xyz_alt = xyz[firstRamp:, :]

lib.so.backProjectionOmpGroundRange( x, y, r_over,
                                    sr.reshape(sr.size),
                                    xyz_alt.reshape(xyz_alt.size), 
                                    imgGroundRange.reshape(imgGroundRange.size),
                                    myParameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

img.shape = (601, 501)
execution time = 518.7463481426239


In [29]:
min_dB = np.amin( 20 * np.log10(np.abs(imgGroundRange)) )
max_dB = np.amax( 20 * np.log10(np.abs(imgGroundRange)) )
med_dB = np.median( 20 * np.log10(np.abs(imgGroundRange)) )
print("min_dB = {:.2f}, max_dB = {:.2f}, med_dB = {:.2f}".format(min_dB, max_dB, med_dB))

min_dB = -54.58, max_dB = 43.42, med_dB = 9.66


In [30]:
opt = OPTt()

opt.db = 1
opt.mod = 0
opt.med_dyn = 30
opt.title = data_date + \
"\nd_x={} d_y={} elevation={} phi={}\n".format( d_x, d_y, hScene, myParameters.phi_a_deg )

disp.disp_PoSAR_img( imgGroundRange.T, opt, x, 0, im_extent, cmap = 'jet' )

plt.plot( runaway_proj[:,0], runaway_proj[:,1], 'k' )
plt.plot( hangar_proj[:,0], hangar_proj[:,1], 'k' )
plt.plot( building_proj[:,0], building_proj[:,1], 'k' )

plt.plot( track_selection_proj[:,0], track_selection_proj[:,1], 'Dy', markeredgecolor='k' )

plt.grid()

db 1
mod 0
med_dyn 30
cmap jet
title 2018_05_29_15_09_37
d_x=1.0 d_y=1.0 elevation=90 phi=20.0



In [31]:
cmap = 'gray'

In [32]:
plt.figure()

plt.imshow( 20 * np.log10( box_filter( np.abs( imgGroundRange.T ), 2 ) ), cmap=cmap )
#plt.plot( x[2195], y[258], 'Dr', markeredgecolor='k' )
#plt.plot( x[2357], y[211], 'Dg', markeredgecolor='k' )

#plt.plot( runaway_proj[:,0], runaway_proj[:,1], 'k' )
#plt.plot( hangar_proj[:,0], hangar_proj[:,1], 'k' )
#plt.plot( building_proj[:,0], building_proj[:,1], 'k' )

plt.colorbar()
ax = plt.gca()
ax.invert_xaxis()
ax.xaxis.tick_top()
ax.yaxis.tick_right()

im.shape = (501, 601)


In [105]:
Nsmooth = 3
orig = np.abs( imgGroundRange.T )
other = box_filter( np.abs( imgGroundRange.T ), Nsmooth )

im.shape = (501, 601)


In [117]:
#cv2.GaussianBlur(frame, image, cv::Size(0, 0), 3)
#cv2.addWeighted(frame, 1.5, image, -0.5, 0, image)

kernel = np.ones( (Nsmooth,Nsmooth) ) / (Nsmooth*Nsmooth)
kernelSharpen = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
kernelEdgeDetection = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])

orig_kernel              = cv2.filter2D( orig, -1, kernel )
orig_kernelSharpen       = cv2.filter2D( orig, -1, kernelSharpen )
orig_kernelEdgeDetection = cv2.filter2D( orig, -1, kernelEdgeDetection )

orig_bilateralFilter = cv2.bilateralFilter( np.float32(orig), 9, 75, 75 )

mask = cv2.GaussianBlur( orig, (5,5), 0 )
alpha = 0.7
img2 = cv2.addWeighted( orig, alpha, mask, 1-alpha, 0 )

#blur = cv2.medianBlur(img, 5 )

# sharpen image using "unsharp mask" algorithm
#Mat blurred; double sigma = 1, threshold = 5, amount = 1;
#GaussianBlur(img, blurred, Size(), sigma, sigma);
#Mat lowContrastMask = abs(img - blurred) < threshold;
#Mat sharpened = img*(1+amount) + blurred*(-amount);
#img.copyTo(sharpened, lowContrastMask);

In [118]:
cmap='gray'

plt.figure()
plt.subplot(221)
ax = plt.gca()
plt.imshow( 20 * np.log10( other ), cmap = cmap )
plt.gca().invert_xaxis()

plt.subplot(222, sharex=ax, sharey=ax)
plt.imshow( 20 * np.log10( orig_kernel ), cmap = cmap )
plt.gca().invert_xaxis()

plt.subplot(223, sharex=ax, sharey=ax)
plt.imshow( 20 * np.log10( orig_kernelSharpen ), cmap = cmap )
plt.gca().invert_xaxis()

plt.subplot(224, sharex=ax, sharey=ax)
plt.imshow( 20 * np.log10( orig_bilateralFilter ), cmap = cmap )
plt.gca().invert_xaxis()

/opt/Py3-6-Venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log10
  


In [108]:
plt.figure()
plt.plot(other[100,:])
plt.plot(orig_kernel[100,:])

# Save focused image

In [ ]:
if groundRange == 1: # [0] ground range, [1] slant range
    if rampUp == 1:
        focusedImageFilename = "/" + data_date \
        + " {} {} GR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampUp alt xyz".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
    else:
        focusedImageFilename = "/" + data_date \
        + " {} {} GR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampDown".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
else:
    if rampUp == 1:
        focusedImageFilename = "/" + data_date \
        + " {} {} SR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampUp".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
    else:
        focusedImageFilename = "/" + data_date \
        + " {} {} SR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampDown".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
    
if withHanning:
    focusedImageFilename + " Hann"
    
print( focusedImageFilename )

In [ ]:
np.save( data_dir + focusedImageFilename, imgGroundRange )

# Save image as png

In [ ]:
plt.imsave( data_dir + focusedImageFilename + ".png",
           20 * np.log10( box_filter( np.abs( np.flip( imgGroundRange.T, 1 ) ), 5 ) ), 
           cmap="gray" )